[Пример с MNIST](https://colab.research.google.com/github/rpi-techfundamentals/fall2018-materials/blob/master/10-deep-learning/04-pytorch-mnist.ipynb#scrollTo=LOLocpDXq3nx)

In [ ]:
!pip install onnxruntime==1.9.0
!pip install torch torchvision
!pip install onnx

In [ ]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import numpy as np


import torch.onnx
import onnxruntime
from sklearn.metrics import classification_report as Report
from onnxruntime.quantization import quantize_dynamic

## MNIST

In [ ]:
args={}
kwargs={}
args['batch_size']=1000
args['test_batch_size']=1000
args['epochs']=3  #The number of Epochs is the number of times you go through the full dataset. 
args['lr']=0.01 #Learning rate is how fast it will decend. 
args['momentum']=0.5 #SGD momentum (default: 0.5) Momentum is a moving average of our gradients (helps to keep direction).

args['seed']=1 #random seed
args['log_interval']=10
args['cuda']=False

In [ ]:
#load the data
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['batch_size'], shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args['test_batch_size'], shuffle=True, **kwargs)

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:180.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [ ]:
class Net(nn.Module):
    #This defines the structure of the NN.
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()  #Dropout
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        #Convolutional Layer/Pooling Layer/Activation
        x = F.relu(F.max_pool2d(self.conv1(x), 2)) 
        #Convolutional Layer/Dropout/Pooling Layer/Activation
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        #Fully Connected Layer/Activation
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        #Fully Connected Layer/Activation
        x = self.fc2(x)
        #Softmax gets probabilities. 
        return F.log_softmax(x, dim=1)

In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        #Variables in Pytorch are differenciable. 
        data, target = Variable(data), Variable(target)
        #This will zero out the gradients for this batch. 
        optimizer.zero_grad()
        output = model(data)
        # Calculate the loss The negative log likelihood loss. It is useful to train a classification problem with C classes.
        loss = F.nll_loss(output, target)
        #dloss/dx for every Variable 
        loss.backward()
        #to do a one-step update on our parameter.
        optimizer.step()
        #Print out the loss periodically. 
        if batch_idx % args['log_interval'] == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(epoch, batch_idx * len(data), 
                                                                           len(train_loader.dataset), 
                                                                           100. * batch_idx / len(train_loader), loss.data))

def test():
    model.eval()
    test_loss = 0
    correct = 0
    for data, target in test_loader:
        if args['cuda']:
            data, target = data.cuda(), target.cuda()
        data, target = Variable(data, volatile=True), Variable(target)
        output = model(data)
        test_loss += F.nll_loss(output, target, size_average=False).data # sum up batch loss
        pred = output.data.max(1, keepdim=True)[1] # get the index of the max log-probability
        correct += pred.eq(target.data.view_as(pred)).long().cpu().sum()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),  
        100. * correct / len(test_loader.dataset)))

In [ ]:
model = Net()
if args['cuda']:
    model.cuda()

optimizer = optim.SGD(model.parameters(), lr=args['lr'], momentum=args['momentum'])

for epoch in range(1, args['epochs'] + 1):
    train(epoch)
    test()


/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Train Epoch: 1 [0/60000 (0%)]	Loss: 2.334774
Train Epoch: 1 [10000/60000 (17%)]	Loss: 2.312112
Train Epoch: 1 [20000/60000 (33%)]	Loss: 2.298412
Train Epoch: 1 [30000/60000 (50%)]	Loss: 2.284640
Train Epoch: 1 [40000/60000 (67%)]	Loss: 2.286018
Train Epoch: 1 [50000/60000 (83%)]	Loss: 2.272294


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:30: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 2.2452, Accuracy: 3607/10000 (36%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 2.252806
Train Epoch: 2 [10000/60000 (17%)]	Loss: 2.248607
Train Epoch: 2 [20000/60000 (33%)]	Loss: 2.224671
Train Epoch: 2 [30000/60000 (50%)]	Loss: 2.204084
Train Epoch: 2 [40000/60000 (67%)]	Loss: 2.164259
Train Epoch: 2 [50000/60000 (83%)]	Loss: 2.124230

Test set: Average loss: 1.9436, Accuracy: 6257/10000 (63%)

Train Epoch: 3 [0/60000 (0%)]	Loss: 2.047359
Train Epoch: 3 [10000/60000 (17%)]	Loss: 1.938156
Train Epoch: 3 [20000/60000 (33%)]	Loss: 1.841257
Train Epoch: 3 [30000/60000 (50%)]	Loss: 1.711682
Train Epoch: 3 [40000/60000 (67%)]	Loss: 1.621157
Train Epoch: 3 [50000/60000 (83%)]	Loss: 1.537818

Test set: Average loss: 1.0572, Accuracy: 7959/10000 (80%)



# Конвертирование в ONNX

Экспорт с помощью трассировки(для информации о запускаемых операциях), для запуска трассировки нужен вход

In [ ]:
bs = 5
inp = torch.randn(bs, 1, 28, 28, requires_grad=True)

In [ ]:
inp.shape

torch.Size([5, 1, 28, 28])

In [ ]:
model(inp)

tensor([[-2.4647, -2.4803, -1.8364, -2.1460, -2.4728, -2.3880, -1.8757, -2.9909,
         -2.3285, -2.5535],
        [-1.9438, -2.9372, -2.2128, -2.3945, -2.3803, -2.2351, -2.0445, -2.5336,
         -2.2964, -2.3615],
        [-2.2160, -2.2806, -2.0913, -1.8523, -3.2853, -2.1227, -2.9352, -2.3848,
         -2.0278, -2.5989],
        [-2.0939, -2.4482, -1.8822, -1.9850, -2.8791, -2.3515, -2.4885, -2.5497,
         -2.1527, -2.6315],
        [-1.9028, -3.0507, -2.1980, -2.5940, -2.1863, -2.2174, -2.0641, -2.5649,
         -2.3679, -2.3123]], grad_fn=<LogSoftmaxBackward>)

In [ ]:
torch.onnx.export(model, inp, 'mnist_torch.onnx', input_names=['input_28_28'], output_names=['output_log_softmax'],
                   dynamic_axes={'input_28_28' : {0 : 'batch_size'},    #  Динамические оси, для переменного размера
                                 'output_log_softmax' : {0 : 'batch_size'}},
                  opset_version=10 # Версия с поддержкой квантования, до 9 не поддерживается
                  )

## Тестирование

In [ ]:
sess = onnxruntime.InferenceSession('mnist_torch.onnx')

In [ ]:
inputs = sess.get_inputs()
for inp in inputs:
  print(inp.name)

input_28_28


In [ ]:
def OnnxTest(x_test, y_test, sess):
  input_name = sess.get_inputs()[0].name
  output_name = sess.get_outputs()[0].name
  outp = sess.run([output_name], {input_name: x_test})

  class_pred = []
  for prob in outp[0]:
    class_pred.append(np.argmax(prob))

  class_real = []
  for real in y_test:
    class_real.append(real)

  print(Report(class_real, class_pred))

In [ ]:
data = test_loader.dataset
x_test = data.train_data.reshape(data.train_data.shape[0], 1, 28,28).numpy().astype(np.float32)

y_test = data.train_labels

/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:62: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")
/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:52: UserWarning: train_labels has been renamed targets
  warnings.warn("train_labels has been renamed targets")


In [ ]:
OnnxTest(x_test, y_test, sess)

              precision    recall  f1-score   support

           0       0.85      0.97      0.91       980
           1       0.93      0.96      0.95      1135
           2       0.80      0.87      0.84      1032
           3       0.72      0.83      0.77      1010
           4       0.77      0.81      0.79       982
           5       0.85      0.49      0.62       892
           6       0.79      0.90      0.84       958
           7       0.76      0.87      0.81      1028
           8       0.77      0.64      0.70       974
           9       0.82      0.64      0.72      1009

    accuracy                           0.80     10000
   macro avg       0.81      0.80      0.79     10000
weighted avg       0.81      0.80      0.80     10000



## Квантование

In [ ]:
quantize_dynamic('/content/mnist_torch.onnx', '/content/mnist_torch_q.onnx')

In [ ]:
sess_q = onnxruntime.InferenceSession('mnist_torch_q.onnx')

In [ ]:
OnnxTest(x_test, y_test, sess_q)

              precision    recall  f1-score   support

           0       0.85      0.97      0.91       980
           1       0.93      0.96      0.95      1135
           2       0.80      0.87      0.83      1032
           3       0.73      0.83      0.78      1010
           4       0.77      0.80      0.79       982
           5       0.84      0.49      0.62       892
           6       0.78      0.90      0.84       958
           7       0.76      0.87      0.81      1028
           8       0.77      0.64      0.70       974
           9       0.82      0.65      0.72      1009

    accuracy                           0.80     10000
   macro avg       0.81      0.80      0.79     10000
weighted avg       0.81      0.80      0.80     10000



In [ ]:
%ls -l --block-size=KB

total 127kB
-rw-r--r-- 1 root root 89kB Oct  4 11:51 mnist_torch.onnx
-rw-r--r-- 1 root root 30kB Oct  4 11:52 mnist_torch_q.onnx
drwxr-xr-x 1 root root  5kB Sep 30 17:12 sample_data/
